# Problem Statement
3 Pollutants:
- Dust and soot
- Sulphur oxides (SxOy)
- Hydrocarbons (e.g. methane)

Assumption: Steel mill reduce annual emisisons in each of the 3 categories:
- Dust and soot: 60 000 tons
- Sulphur oxides: 150 000 tons
- Hydrocarbons: 125 000 tons

2 main sources of emissions:
- Type 1: Blast furnace for the production of pig iron.
- Type 2: Blast furnace for converting into steel.

Measures:
- Raising the chimneys
- Installation of filters in the chimneys
- Use of cleaner fuels with a higher calorific value

--> Can be used for one or both blast furnaces, therefore a total of 6 measures possible.


Reduction of pollutats by measure (in 1000 tons):
| Pollutant         | Chimneys T1 | Chimneys T2| Filter T1 | Filter T2| Fuel T1 | Fuel T2|
|--------------------|----------------|---------------|----------|---------|---------------|--------------|
| Dust and Soot      | 12             | 9             | 25       | 20      | 17            | 13           |
| Sulfuroxides      | 35             | 42            | 18       | 31      | 56            | 49           |
| Hydrocarbons | 37             | 53            | 28       | 24      | 29            | 20           |

Costs of the measures (in EUR million):

| Measure | Type 1 | Type 2|
|---------|--------|-------|
|Chimneys |8       |10     |
|Filter   |7       |6      |
|Fuel     |11      |9      |

Each of the measures can be implemented as desired on a scale of 0-100%. The costs for the measures are weighted with the degree of implementation of the respective measure.

a) Set up a mathematical model of the optimization problem (objective function, constraints) that minimizes the costs incurred while complying with the required emission reductions.

b) Solve the optimization problem with the help of Python.

### Decision Variables:
Let $x_{ij}$ be the fraction (from 0 to 1) to which measure $i$ is implemented on furnace type $j$.

Where $i$ can be:
- $c$ for Chimneys
- $f$ for Filter
- $u$ for Fuel

And $j$ can be:
- 1 for Type 1 furnace
- 2 for Type 2 furnace

### Objective Function:
Minimize the total cost $C:C = 8x_{c1} + 10x_{c2} + 7x_{f1} + 6x_{f2} + 11x_{u1} + 9x_{u2}$

### Constraints:
The reductions for each pollutant must meet or exceed the target reductions. Let $r_{ij}$ be the reduction of pollutant by measure $i$ on furnace type $j$, given in thousands of tons.

#### For Dust and Soot (DS):
$12x_{c1} + 9x_{c2} + 25x_{f1} + 20x_{f2} + 17x_{u1} + 13x_{u2} \ge 60$

#### For Sulfur Oxides (SO):
$35x_{c1} + 42x_{c2} + 18x_{f1} + 31x_{f2} + 56x_{u1} + 49x_{u2} \ge 150$

#### For Hydrocarbons (HC):
$37x_{c1} + 53x_{c2} + 28x_{f1} + 24x_{f2} + 29x_{u1} + 20x_{u2} \ge 125$

### Lower Bound & Upper Bound:
Each $x_{ij}$ is bounded by the implementation scale:
$0 \le x_{ij} \le 1$ for all $i$ in ${c, f, u}$, $j$ in ${1, 2}$


In [11]:
from pulp import LpProblem, LpMinimize, LpVariable, lpSum

# Define the problem
prob = LpProblem("Pollution_Control", LpMinimize)

# Decision variables
x = LpVariable.dicts("x", [(i,j) for i in ['c', 'f', 'u'] for j in [1, 2]], 0, 1)

# Objective function
prob += 8*x[('c',1)] + 10*x[('c',2)] + 7*x[('f',1)] + 6*x[('f',2)] + 11*x[('u',1)] + 9*x[('u',2)], "Total_Cost"

# Constraints
prob += 12*x[('c',1)] + 9*x[('c',2)] + 25*x[('f',1)] + 20*x[('f',2)] + 17*x[('u',1)] + 13*x[('u',2)] >= 60, "Dust_and_Soot"
prob += 35*x[('c',1)] + 42*x[('c',2)] + 18*x[('f',1)] + 31*x[('f',2)] + 56*x[('u',1)] + 49*x[('u',2)] >= 150, "Sulfur_Oxides"
prob += 37*x[('c',1)] + 53*x[('c',2)] + 28*x[('f',1)] + 24*x[('f',2)] + 29*x[('u',1)] + 20*x[('u',2)] >= 125, "Hydrocarbons"

# Solve the problem
prob.solve()

n = prob.variables()
n[0].name = 'Chimney Type 1'
n[1].name = 'Chimney Type 2'
n[2].name = 'Filter Type 1'
n[3].name = 'Filter Type 2'
n[4].name = 'Fuel Type 1'
n[5].name = 'Fuel Type 2'

# Print the results
print("The optimal values of the decision variables:")
for v in prob.variables():
    print(f"{v.name} = {v.varValue:.2f}")
print(f"Total cost = {prob.objective.value():.2f} million EUR")


The optimal values of the decision variables:
Chimney_Type_1 = 1.00
Chimney_Type_2 = 0.62
Filter_Type_1 = 0.34
Filter_Type_2 = 1.00
Fuel_Type_1 = 0.05
Fuel_Type_2 = 1.00
Total cost = 32.15 million EUR


The assumption from point a) that the measures can be implemented to any degree is not easy to implement in reality. Therefore, the following task is now to be solved:

c) Convert the LP from point a) into an MILP, i.e. each of the 6 measures can either be implemented 100% or not at all.

d) Only one type (type 1 or type 2) of each of the measures (chimneys, filters, fuels) can be implemented. Expand the MILP to include these secondary conditions and solve it in software. Can the required reductions be achieved at all?

### Decision Variables:
Let $x_{ij}$ be binary variables such that:
- $x_{ij} = 1$ if measure $i$ is fully implemented on furnace type $j$,
- $x_{ij} = 0$ if measure $i$ is not implemented on furnace type $j$.

### Objective Function:
Minimize the total cost $C: C = 8x_{c1} + 10x_{c2} + 7x_{f1} + 6x_{f2} + 11x_{u1} + 9x_{u2}$

### Constraints:
The reductions for each pollutant must meet or exceed the target reductions:

#### For Dust and Soot (DS):
$12x_{c1} + 9x_{c2} + 25x_{f1} + 20x_{f2} + 17x_{u1} + 13x_{u2} >= 60$

#### For Sulfur Oxides (SO):
$35x_{c1} + 42x_{c2} + 18x_{f1} + 31x_{f2} + 56x_{u1} + 49x_{u2} >= 150$

#### For Hydrocarbons (HC):
$37x_{c1} + 53x_{c2} + 28x_{f1} + 24x_{f2} + 29x_{u1} + 20x_{u2} >= 125$

### Binary Variable Constraint:
Each $x_{ij}$ is a binary integer variable:
$x_{ij}$ in ${0, 1}$ for all $i$ in ${c, f, u}$, $j$ in ${1, 2}$


In [15]:
from pulp import LpProblem, LpMinimize, LpVariable, lpSum, LpBinary

# Define the problem
prob = LpProblem("Pollution_Control", LpMinimize)

# Decision variables
x = LpVariable.dicts("x", [(i,j) for i in ['c', 'f', 'u'] for j in [1, 2]], 0, 1, cat=LpBinary)

# Objective function
prob += 8*x[('c',1)] + 10*x[('c',2)] + 7*x[('f',1)] + 6*x[('f',2)] + 11*x[('u',1)] + 9*x[('u',2)], "Total_Cost"

# Constraints
prob += 12*x[('c',1)] + 9*x[('c',2)] + 25*x[('f',1)] + 20*x[('f',2)] + 17*x[('u',1)] + 13*x[('u',2)] >= 60, "Dust_and_Soot"
prob += 35*x[('c',1)] + 42*x[('c',2)] + 18*x[('f',1)] + 31*x[('f',2)] + 56*x[('u',1)] + 49*x[('u',2)] >= 150, "Sulfur_Oxides"
prob += 37*x[('c',1)] + 53*x[('c',2)] + 28*x[('f',1)] + 24*x[('f',2)] + 29*x[('u',1)] + 20*x[('u',2)] >= 125, "Hydrocarbons"

# Solve the problem
prob.solve()

n = prob.variables()
n[0].name = 'Chimney Type 1'
n[1].name = 'Chimney Type 2'
n[2].name = 'Filter Type 1'
n[3].name = 'Filter Type 2'
n[4].name = 'Fuel Type 1'
n[5].name = 'Fuel Type 2'

# Print the results
print("The optimal values of the decision variables:")
for v in prob.variables():
    print(v.name, "=", v.varValue)
print(f"Total cost = {prob.objective.value():.2f} million EUR")


The optimal values of the decision variables:
Chimney_Type_1 = 1.0
Chimney_Type_2 = 1.0
Filter_Type_1 = 1.0
Filter_Type_2 = 0.0
Fuel_Type_1 = 1.0
Fuel_Type_2 = 0.0
Total cost = 36.00 million EUR


### Additional Constraints for Measure Type Limitation:
Each measure is implemented on only one type of furnace:

#### For Chimneys:
$x_{c1} + x_{c2} \le 1$

#### For Filters:
$x_{f1} + x_{f2} \le 1$

#### For Fuel:
$x_{u1} + x_{u2} \le 1$

In [23]:
from pulp import LpProblem, LpMinimize, LpVariable, lpSum, LpBinary

# Define the problem
prob = LpProblem("Pollution_Control", LpMinimize)

# Decision variables
x = LpVariable.dicts("x", [(i,j) for i in ['c', 'f', 'u'] for j in [1, 2]], 0, 1, cat=LpBinary)

# Objective function
prob += 8*x[('c',1)] + 10*x[('c',2)] + 7*x[('f',1)] + 6*x[('f',2)] + 11*x[('u',1)] + 9*x[('u',2)], "Total_Cost"

# Constraints
prob += 12*x[('c',1)] + 9*x[('c',2)] + 25*x[('f',1)] + 20*x[('f',2)] + 17*x[('u',1)] + 13*x[('u',2)] >= 60, "Dust_and_Soot"
prob += 35*x[('c',1)] + 42*x[('c',2)] + 18*x[('f',1)] + 31*x[('f',2)] + 56*x[('u',1)] + 49*x[('u',2)] >= 150, "Sulfur_Oxides"
prob += 37*x[('c',1)] + 53*x[('c',2)] + 28*x[('f',1)] + 24*x[('f',2)] + 29*x[('u',1)] + 20*x[('u',2)] >= 125, "Hydrocarbons"
prob += x[('c',1)] + x[('c',2)] <= 1, "Chimneys"
prob += x[('f',1)] + x[('f',2)] <= 1, "Filters"
prob += x[('u',1)] + x[('u',2)] <= 1, "Fuel"

# Solve the problem
prob.solve()

n = prob.variables()
n[0].name = 'Chimney Type 1'
n[1].name = 'Chimney Type 2'
n[2].name = 'Filter Type 1'
n[3].name = 'Filter Type 2'
n[4].name = 'Fuel Type 1'
n[5].name = 'Fuel Type 2'

# Print the results
print("The optimal values of the decision variables:")

for v in prob.variables():
    print(v.name, "=", v.varValue)
print("Total cost =", prob.objective.value())

The optimal values of the decision variables:
Chimney_Type_1 = 0.0
Chimney_Type_2 = 1.0
Filter_Type_1 = 1.1666667
Filter_Type_2 = 1.0
Fuel_Type_1 = 1.0
Fuel_Type_2 = 0.0
Total cost = 35.166666899999996
